In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Deliverable2").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [4]:
vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_table.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|                Y|
|R3QWMLJHIW6P37|          5|            0|          0|   N|                Y|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|                Y|
| R25ZRJL0GH0U0|          2|            0|          0|   N|                Y|
|R3837KYH7AZNIY|          4|            0|          1|   N|                Y|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|                Y|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|                Y|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|                Y|
|R16ZDMJJHK796C|          5|            0|          0|   N|                N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|     

In [5]:
vine_df=vine_table.filter(vine_table.total_votes >=20)
vine_df.orderBy(vine_df['total_votes'].asc()).show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VI75W9GLU0UE|          1|            5|         20|   N|                Y|
|R1GMRTTRZOR2JQ|          2|           20|         20|   N|                Y|
|R35X1N5KF93TC8|          1|            2|         20|   N|                Y|
|R28VFN6VN0VBTB|          1|            7|         20|   N|                N|
|R38S6X7GNDUMD4|          5|           18|         20|   N|                Y|
| R9F7EN48ULXMD|          1|           11|         20|   N|                Y|
|R3C2TJDN9Q5S67|          3|            3|         20|   N|                Y|
|R1GHR6F1XRBNAZ|          1|           10|         20|   N|                Y|
| RVNEUP25G4NWN|          4|            6|         20|   N|                N|
|R3NROF8024WIF7|          5|           18|         20|   N|     

In [29]:
vine_helpful_votes=vine_df.filter(vine_df.helpful_votes/vine_df.total_votes >= 0.5)
vine_helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [30]:
vine_review=vine_helpful_votes.filter(vine_helpful_votes.vine=='Y')
vine_review.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R2RHQWEKX5NL|          5|           81|         89|   Y|                N|
|R1RZ4JZO8DAP7H|          3|          200|        227|   Y|                N|
| RQ94HI0WM8KIG|          5|          174|        188|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [32]:
nonvine_review=vine_helpful_votes.filter(vine_helpful_votes.vine=='N')
nonvine_review.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [33]:
#total number of reviews
total_num_reviews=vine_helpful_votes.count()
total_num_reviews

3097

In [35]:
#total number of Vine reviews 
total_vine_reviews=vine_review.count()
total_vine_reviews

3

In [36]:
#total number of non vine reviews
total_nonvine_reviews=nonvine_review.count()
total_nonvine_reviews

3094

In [37]:
#total 5-star reviews
five_star_reviews=vine_helpful_votes.filter(vine_helpful_votes.star_rating=="5")
five_star_reviews.count()

1706

In [39]:
#total 5-star reviews of Vine reviews 
five_star_vine_reviews=vine_review.filter(vine_review.star_rating=="5")
five_star_vine_reviews.count()

2

In [40]:
#total 5-star reviews of NON Vine reviews
five_star_NONvine_reviews=nonvine_review.filter(nonvine_review.star_rating=="5")
five_star_NONvine_reviews.count()

1704

In [45]:
#% of 5-star Vine reviews
percentage_five_star_Vine= (five_star_vine_reviews.count()/five_star_reviews.count())*100

round(percentage_five_star_Vine,2)

0.12

In [46]:
#% of 5-star NON Vine reviews
percentage_five_star_NONVine= (five_star_NONvine_reviews.count()/five_star_reviews.count())*100

round(percentage_five_star_NONVine,2)

99.88